In [1]:
import pandas as pd
import numpy as np
import os
import subprocess

In [2]:
# pip install wget

In [3]:
df = pd.read_csv('./resource/phenotypes_202304020100.csv', sep=';')

In [4]:
df.head()
df.shape

(6674, 719)

In [5]:
avail = df[df["Colour Blindness"]!='-']

In [6]:
avail['Colour Blindness'].unique()

array(['False', 'Not color blind', 'Not colour blind', 'not color blind',
       'Red-green colour blind', 'Mild red-green colour blind',
       'Mild blue-black colour blind',
       'low light blue green colour blindness',
       'Low light blue green colour blindness',
       'trouble distinguishing between dark blue & black (not color blindness), dad had red-green color blindness',
       'No', 'carrier (eg: my brother and my son have colour blindness)',
       'Carrier (eg: my brother and my son have colour blindness)',
       'Not colour blind, but difficulty telling red/orange/pink apart sometimes',
       'Tritanomaly',
       'Trouble distinguishing purples, dark blue, violets. ',
       'Moderate to Strong Protanopia',
       'Not color blind, but my parent is color blind', nan, 'N/a', 'Yes',
       'slight blue, purple'], dtype=object)

## Two ways to seperate phenotypes:

1. Yes/No Color-Blindness

Yes: ['Red-green colour blind', 'Mild red-green colour blind', 'low light blue green colour blindness', 'Low light blue green colour blindness', 'Tritanomaly', 'Moderate to Strong Protanopia', 'Yes', 'slight blue, purple', 'Trouble distinguishing purples, dark blue, violets. ']

No: ['False', 'Not color blind', 'Not colour blind', 'not color blind', 'Not color blind, but my parent is color blind', 'trouble distinguishing between dark blue & black (not color blindness), dad had red-green color blindness',
       'No', 'carrier (eg: my brother and my son have colour blindness)',
       'Carrier (eg: my brother and my son have colour blindness)',
       'Not colour blind, but difficulty telling red/orange/pink apart sometimes',]

2. Type of Color-Blindess

Red-green: ['Red-green colour blind', 'Mild red-green colour blind', 'Moderate to Strong Protanopia']

Blue-yellow: ['low light blue green colour blindness', 'Low light blue green colour blindness', 'Tritanomaly', 'slight blue, purple', 'Trouble distinguishing purples, dark blue, violets. ']

In [7]:
avail['user_id'] #there are individuals with multiple files

0           1
1           1
2           1
3           1
5           8
        ...  
6592    11641
6593    11643
6622    11698
6624    11700
6650    11753
Name: user_id, Length: 785, dtype: int64

In [8]:
avail.shape

(785, 719)

In [9]:
avail

,user_id,genotype_filename,date_of_birth,chrom_sex,openhumans_name,Eye color,Lactose intolerance,Handedness,white skin,Ability to find a bug in openSNP,...,Dementia Resistance,Short Stature,Root resorption of tooth,Congenital adrenal hyperplasia (CAH),Difficulty reaching climax/ orgasming with partner,Thin Skin,Excessive sweating from heat (Eccrine sweat glands),Emotional sweating (apocrine sweat glands),Cold-Induced Sweating,"Epicanthic Fold, Hooded Eyes"
0,1,1.23andme.9,1985,XY,-,blue-green,lactose-tolerant,left-handed,Caucasian,Only creating bugs,...,-,-,-,-,-,-,-,-,-,-
1,1,1.23andme.2995,1985,XY,-,blue-green,lactose-tolerant,left-handed,Caucasian,Only creating bugs,...,-,-,-,-,-,-,-,-,-,-
2,1,1.23andme-exome-vcf.3302,1985,XY,-,blue-green,lactose-tolerant,left-handed,Caucasian,Only creating bugs,...,-,-,-,-,-,-,-,-,-,-
3,1,1.23andme.5299,1985,XY,-,blue-green,lactose-tolerant,left-handed,Caucasian,Only creating bugs,...,-,-,-,-,-,-,-,-,-,-
5,8,8.23andme.2,rather not say,XX,-,Brown,lactose-intolerant,left-handed,True,c-c-combo breaker!,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,11641,11641.ftdna-illumina.9569,1990,XY,Daiten,Green-gray,-,"Right-handed dominant, left-handed for eating",-,-,...,-,-,-,-,-,-,-,-,-,-
6593,11643,11643.23andme.9571,rather not say,rather not say,-,Brown,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6622,11698,11698.23andme.9613,1997,XY,takiwatanga,Brown-green,Partially lactose intolerant - surfaces with a...,Left-handed,Caucasian,Positive,...,-,-,Yes,suspected diagnosis,Almost Impossible,-,Yes,High,Yes,-
6624,11700,11700.ancestry.9615,rather not say,rather not say,-,-,-,Right-handed,Caucasian,-,...,-,-,-,-,-,-,-,-,-,-


In [10]:
cb = avail[['user_id', 'genotype_filename', 'date_of_birth', 'chrom_sex', 'Colour Blindness']]
cb = cb.rename(columns={'Colour Blindness':'raw_colourblindness'})
cb = cb.dropna()
cb = cb[cb['raw_colourblindness']!='N/a']
cb.groupby('raw_colourblindness', axis=0).count()['user_id']

raw_colourblindness
Carrier (eg: my brother and my son have colour blindness)                                                      4
False                                                                                                        124
Low light blue green colour blindness                                                                          4
Mild blue-black colour blind                                                                                   4
Mild red-green colour blind                                                                                   24
Moderate to Strong Protanopia                                                                                  1
No                                                                                                            10
Not color blind                                                                                              303
Not color blind, but my parent is color blind                               

In [11]:
yes  = {key:True for key in ['Red-green colour blind', 'Mild red-green colour blind', 'low light blue green colour blindness', 'Low light blue green colour blindness', 'Tritanomaly', 'Moderate to Strong Protanopia', 'Yes', 'Mild blue-black colour blind', 'slight blue, purple', 'Trouble distinguishing purples, dark blue, violets. ']}
no = {key:False for key in ['False', 'Not color blind', 'Not colour blind', 'not color blind', 'Not color blind, but my parent is color blind', 'trouble distinguishing between dark blue & black (not color blindness), dad had red-green color blindness',
       'No', 'carrier (eg: my brother and my son have colour blindness)',
       'Carrier (eg: my brother and my son have colour blindness)',
       'Not colour blind, but difficulty telling red/orange/pink apart sometimes',]}
yes.update(no)
cb['CBDiscrete'] = cb['raw_colourblindness'].map(yes)
cb

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete
0,1,1.23andme.9,1985,XY,False,False
1,1,1.23andme.2995,1985,XY,False,False
2,1,1.23andme-exome-vcf.3302,1985,XY,False,False
3,1,1.23andme.5299,1985,XY,False,False
5,8,8.23andme.2,rather not say,XX,False,False
...,...,...,...,...,...,...
6592,11641,11641.ftdna-illumina.9569,1990,XY,False,False
6593,11643,11643.23andme.9571,rather not say,rather not say,Red-green colour blind,True
6622,11698,11698.23andme.9613,1997,XY,False,False
6624,11700,11700.ancestry.9615,rather not say,rather not say,Not colour blind,False


In [12]:
counts = cb.groupby('user_id').count()
counts

,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete
user_id,,,,,
1,4,4,4,4,4
8,1,1,1,1,1
10,2,2,2,2,2
11,1,1,1,1,1
14,1,1,1,1,1
...,...,...,...,...,...
11641,1,1,1,1,1
11643,1,1,1,1,1
11698,1,1,1,1,1


In [13]:
colorBlind = cb[cb['CBDiscrete']==True]

In [14]:
cb.groupby('CBDiscrete').count()

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness
CBDiscrete,,,,,
False,720,720,720,720,720
True,63,63,63,63,63


In [15]:
cb[cb['user_id']==1]

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete
0,1,1.23andme.9,1985,XY,False,False
1,1,1.23andme.2995,1985,XY,False,False
2,1,1.23andme-exome-vcf.3302,1985,XY,False,False
3,1,1.23andme.5299,1985,XY,False,False


In [16]:
urlHead = "https://opencravat.org/data/open-snp/"

In [17]:
allFiles = np.loadtxt("./resource/genotype_files.txt", dtype=str)

In [18]:
userID = cb['user_id'].unique().tolist()

In [19]:
fileNames = []

for file in allFiles:
    if int(file.split("_")[0][4:]) in userID:
        fileNames.append(file)

In [20]:
cb

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete
0,1,1.23andme.9,1985,XY,False,False
1,1,1.23andme.2995,1985,XY,False,False
2,1,1.23andme-exome-vcf.3302,1985,XY,False,False
3,1,1.23andme.5299,1985,XY,False,False
5,8,8.23andme.2,rather not say,XX,False,False
...,...,...,...,...,...,...
6592,11641,11641.ftdna-illumina.9569,1990,XY,False,False
6593,11643,11643.23andme.9571,rather not say,rather not say,Red-green colour blind,True
6622,11698,11698.23andme.9613,1997,XY,False,False
6624,11700,11700.ancestry.9615,rather not say,rather not say,Not colour blind,False


In [21]:
fileNames

['user10020_file8283_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user10093_file8353_yearofbirth_unknown_sex_unknown.ancestry.txt',
 'user10128_file8398_yearofbirth_1989_sex_XX.23andme.txt',
 'user10188_file8451_yearofbirth_2005_sex_XY.ancestry.txt',
 'user1020_file507_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user10233_file8494_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user10260_file8525_yearofbirth_1995_sex_XX.ancestry.txt',
 'user10260_file8527_yearofbirth_1995_sex_XX.ancestry.txt',
 'user1028_file514_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user1029_file515_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user1035_file518_yearofbirth_1986_sex_XX.ancestry.txt',
 'user10362_file8611_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user10383_file8633_yearofbirth_unknown_sex_unknown.ancestry.txt',
 'user1039_file521_yearofbirth_1958_sex_XY.23andme.txt',
 'user103_file50_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user10408_file8659_yearofbirth_unknown_sex_unknow

In [22]:
def format(fileName:str):
    return fileName.split('.')[1]

In [23]:
cb['format'] = cb['genotype_filename'].transform(format)

In [24]:
file23andMe = cb[cb['format']=='23andme']

In [30]:
file23andMe.shape

(514, 8)

In [25]:
fileNamesSplit = [int(file.split('_')[1][4:]) for file in fileNames]

In [26]:
def mapFileName(genotypeF):
    fileNum = int(genotypeF.split('.')[-1])
    return fileNames[fileNamesSplit.index(fileNum)]

In [27]:
file23andMe['fileName'] = file23andMe['genotype_filename'].transform(mapFileName)

C:\Users\175le\AppData\Local\Temp\ipykernel_23020\4126463386.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file23andMe['fileName'] = file23andMe['genotype_filename'].transform(mapFileName)


In [28]:
def fileSize(file:str):
    return os.path.getsize('./raw/'+file)

In [29]:
file23andMe['fileSize'] = file23andMe['fileName'].transform(fileSize)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not Series

In [ ]:
def build(file:str):
    openFile = open('./raw/'+file, 'rb')
    for i in range(50):
        line = str(openFile.readline()).split(' ')
        if 'build' in line:
            return line[line.index('build')+1][:2]
    return -1

In [ ]:
build('user10020_file8283_yearofbirth_unknown_sex_unknown.23andme.txt')

In [ ]:
builds = [build(file) for file in file23andMe['fileName']]

In [ ]:
file23andMe['builds'] = builds

In [ ]:
file23andMe.groupby('builds').count()

In [ ]:
validFiles = file23andMe[file23andMe['builds']=='37']

In [ ]:
b = validFiles.groupby('user_id').count()

In [ ]:
repeatID = b[b['genotype_filename']>1].index

In [ ]:
repeatRemove = validFiles[validFiles.user_id.isin(repeatID)]

In [ ]:
dropIndex = []
for id in repeatID:
    check = validFiles[validFiles.user_id == id].fileSize
    maxIndex = check.argmax()
    checkIndex = check.index.to_list()
    del checkIndex[maxIndex]
    dropIndex.append(checkIndex)


In [ ]:
drop = [i for sublist in dropIndex for i in sublist]

In [ ]:
final = validFiles.drop(drop)

In [ ]:
final = final.reset_index(drop=True)

In [ ]:
for index, row in final.iterrows():
    command = "./plink --23file ./raw/{filename} FAM{userID} {userID} {sex} {phenotype} --make-bed --out ./processed/{userID}".format(filename=row['fileName'],
                                                                                                                    userID=row['user_id'],
                                                                                                                    sex=row['chrom_sex'],
                                                                                                                    phenotype=row['CBDiscrete'])

In [ ]:
mapSex = {'XX':1, 'XY':2, 'rather not say':'i'}

In [ ]:
final['chrom_sex'] = final['chrom_sex'].map(mapSex)

In [ ]:
mapPhen = {True:2, False:1}
final['CBDiscrete'] = final['CBDiscrete'].map(mapPhen)

In [ ]:
row = final.iloc[1]

In [ ]:
command = "./plink --23file ./raw/{filename} FAM{userID} ID{userID} i {phenotype} --make-bed --out ./processed/{userID}".format(filename=row['fileName'],
                                                                                                                    userID=row['user_id'],
                                                                                                                    phenotype=row['CBDiscrete'])

In [ ]:
for index, row in final.iterrows():
    command = "./plink --23file ./raw/{filename} FAM{userID} ID{userID} i {phenotype} --make-bed --out ./processed/{userID}".format(filename=row['fileName'],
                                                                                                                    userID=row['user_id'],
                                                                                                                    phenotype=row['CBDiscrete'])
    os.system(command)

In [ ]:
processedFiles = os.popen('ls ./processed').read().split('\n')

In [ ]:
errorFiles = [name for name in processedFiles if "temporary" in name]

In [146]:
errorFiles = set([int(file.split('-')[0]) for file in errorFiles])

In [147]:
errorFiles

{3882, 4192, 5211, 5508, 6191, 8522, 8915, 9486, 10632}

In [148]:
final

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete,format,fileName,fileSize,builds
0,1,1.23andme.5299,1985,2,False,1,23andme,user1_file5299_yearofbirth_1985_sex_XY.23andme...,16819733,37
1,10,10.23andme.903,1982,2,False,1,23andme,user10_file903_yearofbirth_1982_sex_XY.23andme...,24755962,37
2,16,16.23andme.3446,1966,2,False,1,23andme,user16_file3446_yearofbirth_1966_sex_XY.23andm...,15683401,37
3,80,80.23andme.864,1979,2,False,1,23andme,user80_file864_yearofbirth_1979_sex_XY.23andme...,24755580,37
4,187,187.23andme.267,1951,2,Not color blind,1,23andme,user187_file267_yearofbirth_1951_sex_XY.23andm...,14786137,37
...,...,...,...,...,...,...,...,...,...,...
359,11056,11056.23andme.9168,1991,2,Mild red-green colour blind,2,23andme,user11056_file9168_yearofbirth_1991_sex_XY.23a...,16786398,37
360,11523,11523.23andme.9416,1988,1,Not colour blind,1,23andme,user11523_file9416_yearofbirth_1988_sex_XX.23a...,15966590,37
361,11531,11531.23andme.9422,1986,2,Yes,2,23andme,user11531_file9422_yearofbirth_1986_sex_XY.23a...,15458410,37
362,11643,11643.23andme.9571,rather not say,i,Red-green colour blind,2,23andme,user11643_file9571_yearofbirth_unknown_sex_unk...,15458816,37


In [149]:
final = final[~final['user_id'].isin(errorFiles)]

In [150]:
final

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete,format,fileName,fileSize,builds
0,1,1.23andme.5299,1985,2,False,1,23andme,user1_file5299_yearofbirth_1985_sex_XY.23andme...,16819733,37
1,10,10.23andme.903,1982,2,False,1,23andme,user10_file903_yearofbirth_1982_sex_XY.23andme...,24755962,37
2,16,16.23andme.3446,1966,2,False,1,23andme,user16_file3446_yearofbirth_1966_sex_XY.23andm...,15683401,37
3,80,80.23andme.864,1979,2,False,1,23andme,user80_file864_yearofbirth_1979_sex_XY.23andme...,24755580,37
4,187,187.23andme.267,1951,2,Not color blind,1,23andme,user187_file267_yearofbirth_1951_sex_XY.23andm...,14786137,37
...,...,...,...,...,...,...,...,...,...,...
359,11056,11056.23andme.9168,1991,2,Mild red-green colour blind,2,23andme,user11056_file9168_yearofbirth_1991_sex_XY.23a...,16786398,37
360,11523,11523.23andme.9416,1988,1,Not colour blind,1,23andme,user11523_file9416_yearofbirth_1988_sex_XX.23a...,15966590,37
361,11531,11531.23andme.9422,1986,2,Yes,2,23andme,user11531_file9422_yearofbirth_1986_sex_XY.23a...,15458410,37
362,11643,11643.23andme.9571,rather not say,i,Red-green colour blind,2,23andme,user11643_file9571_yearofbirth_unknown_sex_unk...,15458816,37


In [151]:
def mergeCommand(ref, target, out):
    return "./plink --bfile ./processed/{ref} --bmerge ./processed/{target} --make-bed --out ./processed/{out} >/dev/null 2>&1".format(ref=ref, target=target, out=out)
def flipCommand(ref, flip):
    return "./plink --bfile ./processed/{ref} --flip ./processed/{flip} --make-bed --out ./processed/{ref}Flip >/dev/null 2>&1".format(ref=ref, flip=flip)
def exCommand(ref, flip):
    return "./plink --bfile ./processed/{ref} --exclude ./processed/{flip} --make-bed --out ./processed/{ref} >/dev/null 2>&1".format(ref=ref, flip=flip)

In [158]:
final['user_id'].to_list().index(4941)

237

In [167]:
final['user_id'].to_list()[352:]

[11531, 11643, 11698]

In [166]:
# os.system(flipCommand("11523miss", "merger-350-merge.missnp"))
# os.system("./plink2 --bfile ./processed/11523 --rm-dup 'exclude-mismatch' --make-bed --out ./processed/11523dup")
# os.system("./plink --bfile ./processed/11523dup --set-missing-var-ids '@:#[b37]\$1,\$2' --make-bed --out ./processed/11523miss")
# os.system(mergeCommand("11523missFlip", "merger-351", "merger-352"))

0

In [168]:
# os.system(mergeCommand("1", "10", "merger-0"))
ref = "merger-352"
count = 353
for user in final['user_id'].to_list()[352:]:
    new = "merger-"+str(count)
    respondID = os.system(mergeCommand(ref, str(user), new))
    
    if respondID != 0:
        os.system(flipCommand(str(user), new+"-merge.missnp"))
        respondID = os.system(mergeCommand(ref, str(user)+"Flip", new))
        if respondID !=0:
            print("second failure")
            os.system(exCommand(ref, new+"-merge.missnp"))
            os.system(exCommand(str(user)+"Flip", new+"-merge.missnp"))
            respondID = os.system(mergeCommand(ref, str(user)+"Flip", new))
            if respondID !=0:
                print("critical failure")
                break
    ref = new
    count += 1


second failure
second failure
second failure


In [73]:
final[final['user_id'] == 11523]

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete,format,fileName,fileSize,builds
360,11523,11523.23andme.9416,1988,1,Not colour blind,1,23andme,user11523_file9416_yearofbirth_1988_sex_XX.23a...,15966590,37


In [138]:
os.system(flipCommand("merger-18", "merger-19-merge.missnp"))

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./processed/merger-18Flip.log.
Options in effect:
  --bfile ./processed/merger-18
  --flip ./processed/merger-19-merge.missnp
  --make-bed
  --out ./processed/merger-18Flip

8192 MB RAM detected; reserving 4096 MB for main workspace.
1518236 variants loaded from .bim file.
20 people (17 males, 3 females) loaded from .fam.
20 phenotype values loaded from .fam.
--flip: 1 SNP flipped.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 20 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.564923.
1518236 variants and 20 people pass filters and QC.
Among remaining

); many commands treat these as missing.


3839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.


0

In [144]:
os.system(exCommand("646Flip", "merger-3-merge.missnp"))

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./processed/646Flip.log.
Options in effect:
  --bfile ./processed/646Flip
  --exclude ./processed/merger-3-merge.missnp
  --make-bed
  --out ./processed/646Flip

8192 MB RAM detected; reserving 4096 MB for main workspace.


filenames.


960613 variants loaded from .bim file.
1 person (1 male, 0 females) loaded from .fam.
1 phenotype value loaded from .fam.
--exclude: 960612 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1 founder and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.994598.
960612 variants and 1 person pass filters and QC.
Among remaining phenotypes, 0 are cases and 1 is a control.
--make-bed to ./processed/646Flip.bed + ./processed/646Flip.bim +
./processed/646Flip.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899%

commands treat these as missing.


done.


0

In [147]:
os.system(mergeCommand("merger-2", "646Flip", "drop"))

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./processed/drop.log.
Options in effect:
  --bfile ./processed/merger-2
  --bmerge ./processed/646Flip
  --make-bed
  --out ./processed/drop

8192 MB RAM detected; reserving 4096 MB for main workspace.
4 people loaded from ./processed/merger-2.fam.
1 person to be merged from ./processed/646Flip.fam.
Of these, 1 is new, while 0 are present in the base dataset.


21 more multiple-position warnings: see log file.
3665 more same-position warnings: see log file.


1488551 markers loaded from ./processed/merger-2.bim.
960612 markers to be merged from ./processed/646Flip.bim.
Of these, 0 are new, while 960612 are present in the base dataset.
Performing single-pass merge (5 people, 1488551 variants).
Merged fileset written to ./processed/drop-merge.bed +
./processed/drop-merge.bim + ./processed/drop-merge.fam .
1488551 variants loaded from .bim file.
5 people (5 males, 0 females) loaded from .fam.
5 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 5 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.549998.
1488551 variants and 5 people pass filters and QC.
Among remaining phenotypes, 0 are cases and 5 are controls.
--make-bed to ./processed/drop.bed 

commands treat these as missing.


69707172737475767778798081828384858687888990919293949596979899done.


0

In [ ]:
# x = os.system(flipCommand("187", "test6-merge.missnp", "187flip"))

In [ ]:
# os.system(mergeCommand("test5", "187flip", "test6"))

In [ ]:
# os.system("./plink --bfile ./processed/10mod --list-duplicate-vars 'suppress-first' 'ids-only' --out ./processed/10mod")

In [ ]:
# os.system("./plink --bfile ./processed/10mod --exclude ./processed/10mod.dupvar --make-bed --out ./processed/10mod2")

In [ ]:
# os.system("./plink --bfile ./processed/10 --snps-only --autosome --make-bed --out ./processed/10mod")

In [ ]:
# os.system("./plink2 --bfile ./processed/test5 --rm-dup 'force-first' 'list' --make-bed --out ./processed/test6")

In [170]:
os.system("./plink --bfile ./processed/merger-355 --recode --out ./processed/recoded")

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./processed/recoded.log.
Options in effect:
  --bfile ./processed/merger-355
  --out ./processed/recoded
  --recode

8192 MB RAM detected; reserving 4096 MB for main workspace.
1561939 variants loaded from .bim file.
355 people (220 males, 135 females) loaded from .fam.
355 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 355 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.


many commands treat these as missing.


Total genotyping rate is 0.462741.
1561939 variants and 355 people pass filters and QC.
Among remaining phenotypes, 21 are cases and 334 are controls.
--recode ped to ./processed/recoded.ped + ./processed/recoded.map ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.


0